第一步：构建训练数据
生成一批随机的序列数据（源序列和目标序列），并对它们做填充操作，使每个序列的长度一致

In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn

#生成一批随机的序列数据（源序列和目标序列），并对它们做填充操作，使每个序列的长度一致
batch_size = 2
#单词表大小
max_num_src_words = 8  # 原序列单词的总数是8
max_num_tgt_words = 8  # 目标序列单词的总数是8
model_dim = 8 #模型的特征大小 原始模型是512 向量维度

# 序列的最大长度
max_src_seq_len = 5  # 定义一个全局的量
max_tgt_seq_len = 5  # 定义一个全局的量
max_position_len =5

src_len = torch.Tensor([2, 4]).to(torch.int32)  # 每个源序列的实际长度
tgt_len = torch.Tensor([4, 3]).to(torch.int32)  # 每个目标序列的实际长度

# 将 `src_len` 和 `tgt_len` 转换为 Python 的整数列表  
src_len = [int(L.item()) for L in src_len]
tgt_len = [int(L.item()) for L in tgt_len]

# 单词索引构成源句子和目标句子，构建batch，并且做了padding,默认值为0
# 对源序列进行填充
src_seq = torch.cat([
    torch.unsqueeze(F.pad(
        torch.randint(1, max_num_src_words, (L,)),  # 创建随机序列，长度为 L
        (0, max_src_seq_len - L)  # 填充到 `max_src_seq_len` 长度
    ),0) for L in src_len
])

# 对目标序列进行填充(原序列变成二维张量)
tgt_seq = torch.cat([
    torch.unsqueeze(F.pad(
        torch.randint(1, max_num_tgt_words, (L,)),  # 创建随机序列，长度为 L
        (0, max_tgt_seq_len - L)  # 填充到 `max_src_seq_len` 长度
    ),0) for L in tgt_len
])

# 打印结果
print(src_seq)
print(tgt_seq)

#以上代码 用单词索引构成源句子和目标句子，并且做了padding,默认值为0

tensor([[5, 1, 0, 0, 0],
        [5, 7, 5, 2, 0]])
tensor([[4, 1, 6, 3, 0],
        [1, 3, 3, 0, 0]])


第二步 构造embedding

构造word embedding

In [9]:
#把文本句子变成一个个数字 （单词在词典中的位置）【第0个位置留给padding]
src_embedding_table = nn.Embedding(max_num_src_words+1,model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1,model_dim)
src_embedding = src_embedding_table(src_seq)#输入的embedding
tgt_embedding = tgt_embedding_table(tgt_seq)#输出的embedding

print(src_embedding_table.weight)
print(src_seq)
print(src_embedding)

Parameter containing:
tensor([[-1.5087,  1.2219,  0.5087, -0.8259,  1.3889, -1.4976,  0.9455,  0.0100],
        [ 0.4668, -1.1043, -0.3958, -0.5707,  1.3332, -0.5401, -0.2981, -1.7680],
        [ 1.8742, -0.6843,  0.0766, -0.4398, -0.1407, -0.8353, -0.5706,  0.1183],
        [-1.1911,  0.1569, -0.8556,  0.5601, -0.7699, -0.5574, -0.9442, -1.0084],
        [ 1.4479, -2.0268,  0.1048,  0.1303,  1.3609,  0.6003, -0.4600, -0.2033],
        [ 0.1169,  0.2863,  0.3168,  0.4440, -0.9636, -1.5183,  1.7822, -0.4421],
        [-0.8873,  0.1426,  0.1239,  0.6848, -1.0032, -0.2704,  1.2433, -1.5858],
        [ 1.6517,  0.3482,  0.8908, -0.3106,  0.9982,  1.2933,  0.4415, -1.4029],
        [-1.6846, -1.0505,  0.9793,  0.8035, -0.0986,  0.8689, -0.0112, -0.6706]],
       requires_grad=True)
tensor([[5, 1, 0, 0, 0],
        [5, 7, 5, 2, 0]])
tensor([[[ 0.1169,  0.2863,  0.3168,  0.4440, -0.9636, -1.5183,  1.7822,
          -0.4421],
         [ 0.4668, -1.1043, -0.3958, -0.5707,  1.3332, -0.5401, -0.2

构造position embedding--位置

In [10]:
pos_mat = torch.arange(max_position_len).reshape((-1,1))
i_mat = torch.pow(10000,torch.arange(0,8,2).reshape((1,-1))/model_dim)
pe_embedding_table = torch.zeros(max_position_len,model_dim)
pe_embedding_table[:,0::2] = torch.sin(pos_mat / i_mat )
print(pe_embedding_table)


print(pe_embedding_table)
pe_embedding = nn.Embedding(max_position_len,model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table,requires_grad=False)

src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len]).to(torch.int32)
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len]).to(torch.int32)

src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)
print(src_pe_embedding)
print(tgt_pe_embedding)
#借助pytorch 的nn 通过位置索引 直接得到位置embedding

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0000,  0.0998,  0.0000,  0.0100,  0.0000,  0.0010,  0.0000],
        [ 0.9093,  0.0000,  0.1987,  0.0000,  0.0200,  0.0000,  0.0020,  0.0000],
        [ 0.1411,  0.0000,  0.2955,  0.0000,  0.0300,  0.0000,  0.0030,  0.0000],
        [-0.7568,  0.0000,  0.3894,  0.0000,  0.0400,  0.0000,  0.0040,  0.0000]])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0000,  0.0998,  0.0000,  0.0100,  0.0000,  0.0010,  0.0000],
        [ 0.9093,  0.0000,  0.1987,  0.0000,  0.0200,  0.0000,  0.0020,  0.0000],
        [ 0.1411,  0.0000,  0.2955,  0.0000,  0.0300,  0.0000,  0.0030,  0.0000],
        [-0.7568,  0.0000,  0.3894,  0.0000,  0.0400,  0.0000,  0.0040,  0.0000]])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8415, 0.0000, 0.0998, 0.0000, 0.0100, 0.0000, 0.0010, 0.0000],
         [0.9093, 0.0000, 0.

In [ ]:
#softmax演示 scaled的重要性
#alpha1 = 0.1#加入参数
#alpha2 = 10
#score = torch.randn(5)#假设有一个向量长度为5 q*k
#prob1 = F.softmax(score * alpha1,-1)#要归一化--为了算注意力机制 用softmax函数
#prob2 = F.softmax(score * alpha2,-1)
#print(score)#一个单词跟整个序列5个单词 他们的相似度的结果
#print(prob1)#prob越大表示这两个单词之间的关联性就越大
#print(prob2)#乘以一个更大的值 score进行缩放 发现不是一个线性

#def softmax_func(score):
    #return F.softmax(score)
#jaco_mat1 = torch.autograd.functional.jacobian(softmax_func,score*alpha1)
#jaco_mat2 = torch.autograd.functional.jacobian(softmax_func,score*alpha2)

#print(jaco_mat1)
#print(jaco_mat2)#很多地方变成了0  没法测试 更新缓慢---QK的方差比价大 所以要除以。。 这样把他的方差弄的小 这样出来的方差就不会尖锐 雅可比矩阵也不会是0 更好拟合模型

tensor([-0.7771, -0.1134, -0.3330, -0.6906,  1.1090])
tensor([0.1876, 0.2005, 0.1961, 0.1892, 0.2265])
tensor([6.4415e-09, 4.9141e-06, 5.4680e-07, 1.5293e-08, 9.9999e-01])
tensor([[ 0.1524, -0.0376, -0.0368, -0.0355, -0.0425],
        [-0.0376,  0.1603, -0.0393, -0.0379, -0.0454],
        [-0.0368, -0.0393,  0.1577, -0.0371, -0.0444],
        [-0.0355, -0.0379, -0.0371,  0.1534, -0.0429],
        [-0.0425, -0.0454, -0.0444, -0.0429,  0.1752]])
tensor([[ 6.4415e-09, -3.1655e-14, -3.5222e-15, -9.8511e-17, -6.4415e-09],
        [-3.1655e-14,  4.9141e-06, -2.6871e-12, -7.5153e-14, -4.9141e-06],
        [-3.5222e-15, -2.6871e-12,  5.4680e-07, -8.3623e-15, -5.4680e-07],
        [-9.8511e-17, -7.5153e-14, -8.3623e-15,  1.5293e-08, -1.5293e-08],
        [-6.4415e-09, -4.9141e-06, -5.4680e-07, -1.5293e-08,  5.4836e-06]])


C:\Users\DELL\AppData\Local\Temp\ipykernel_22604\2085042164.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(score)


In [42]:
import numpy as np

#构造encoder的self-attention mask
#mask shape: [batch_size,max_src_len,max_src_len],值为1或-inf
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max_src_seq_len-L)),0) for L in src_len]),2)#构建有效矩阵 1 代表有效 0代表无效 pad填充 +扩维---有效的编码器位置矩阵 扩到第一维
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1,2))
invalid_encoder_pos_matrix = 1 - valid_encoder_pos_matrix#无效矩阵=1-有效矩阵
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)#变成bool型

#randn(batch_size,max(src_len),max(src_len))
#print(score.shape,mask_encoder_self_attention.shape)

# 示例分数矩阵
score = torch.randn(batch_size, max_src_seq_len, max_src_seq_len)

masked_score = score.masked_fill(mask_encoder_self_attention,-1e9)#10^（-9）
prob = F.softmax(masked_score,-1)


#print(valid_encoder_pos.shape)#每一行都是一个sample 但是sample之间是无关的 所以我们要扩维
#print(valid_encoder_pos_matrix)
#print(invalid_encoder_pos_matrix)
#print(mask_encoder_self_attention)
print(src_len)
print(score)
print(masked_score)
print(prob)

[2, 4]
tensor([[[ 0.1757,  0.3235,  1.4129, -2.4193,  0.5024],
         [-0.0364, -1.1438, -0.5477, -0.4542,  0.2674],
         [ 0.0181,  0.3356,  0.0515,  1.2012, -0.5275],
         [ 1.0760,  0.1892,  0.1757,  0.0074, -0.0313],
         [ 0.9178, -1.0964, -1.3748, -0.4647,  0.5871]],

        [[-0.5212, -0.7235,  1.7528,  2.3971, -0.1821],
         [-0.3758, -0.7509,  0.2712, -1.0049, -1.3839],
         [-1.4135, -0.5290, -0.2983, -0.7322, -1.8117],
         [-0.9284, -0.4624,  1.9035, -0.3858,  0.9875],
         [-0.2232, -0.6820, -1.6235,  0.2576, -0.8895]]])
tensor([[[ 1.7570e-01,  3.2354e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-3.6421e-02, -1.1438e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[-5.2123e-01, -7.2346e-01,  1.7528e+00